In [0]:
from pyspark.sql.functions import lit,col,explode,count,when,split,regexp_replace

In [0]:
columns = ["ID","StudentName","DepartmentName","city","Marks"]
sample_data = [(1,"Nihala","ECE","Salem",99),
                (2,"lishma","CSE","chennai",98),
                (3,"nancy","IT","Nellor",96),
                (4,"vishnu","BSC","TPT",93)]
df_1 = spark.createDataFrame(data=sample_data,schema=columns)
df_1.show()

+---+-----------+--------------+-------+-----+
| ID|StudentName|DepartmentName|   city|Marks|
+---+-----------+--------------+-------+-----+
|  1|     Nihala|           ECE|  Salem|   99|
|  2|     lishma|           CSE|chennai|   98|
|  3|      nancy|            IT| Nellor|   96|
|  4|     vishnu|           BSC|    TPT|   93|
+---+-----------+--------------+-------+-----+



In [0]:
sample_data_2 = [(1,"Deepika","AERO","Salem"),
                 (2,"Sowmi","IT","Madurai")]
columns = ["ID","StudentName","DepartmentName","city"]    
df_2 = spark.createDataFrame(data = sample_data_2,schema = columns)
df_2.show()

+---+-----------+--------------+-------+
| ID|StudentName|DepartmentName|   city|
+---+-----------+--------------+-------+
|  1|    Deepika|          AERO|  Salem|
|  2|      Sowmi|            IT|Madurai|
+---+-----------+--------------+-------+



In [0]:
df_2 = df_2.withColumn("Marks",lit("null"))
df = df_1.union(df_2)
df.show()

+---+-----------+--------------+-------+-----+
| ID|StudentName|DepartmentName|   city|Marks|
+---+-----------+--------------+-------+-----+
|  1|     Nihala|           ECE|  Salem|   99|
|  2|     lishma|           CSE|chennai|   98|
|  3|      nancy|            IT| Nellor|   96|
|  4|     vishnu|           BSC|    TPT|   93|
|  1|    Deepika|          AERO|  Salem| null|
|  2|      Sowmi|            IT|Madurai| null|
+---+-----------+--------------+-------+-----+



In [0]:
sample_data = [(1,["Nihala","Taskeen"]),
               (2,["Mohamed","Rayan"]),
               (3,["Divya","Shree"]),
               (4,["Deepika","Sri"])]
columns = ["ID","Name"]
df = spark.createDataFrame(data = sample_data,schema = columns)
df.show()
df_output = df.select(col("ID"),explode(col("Name")).alias("Name"))
df_output.show()

+---+-----------------+
| ID|             Name|
+---+-----------------+
|  1|[Nihala, Taskeen]|
|  2| [Mohamed, Rayan]|
|  3|   [Divya, Shree]|
|  4|   [Deepika, Sri]|
+---+-----------------+

+---+-------+
| ID|   Name|
+---+-------+
|  1| Nihala|
|  1|Taskeen|
|  2|Mohamed|
|  2|  Rayan|
|  3|  Divya|
|  3|  Shree|
|  4|Deepika|
|  4|    Sri|
+---+-------+



In [0]:
data = [(1,"Nancy","U6946868iy"),
        (2,"Lishma","8148190681"),
        (3,"Sowmi","77089237U8")]
schema = ["ID","Name","Mobile_no"]
df = spark.createDataFrame(data = data,schema = schema)
df.show()

+---+------+----------+
| ID|  Name| Mobile_no|
+---+------+----------+
|  1| Nancy|U6946868iy|
|  2|Lishma|8148190681|
|  3| Sowmi|77089237U8|
+---+------+----------+



In [0]:
df.select("*").filter(col("Mobile_no").rlike('^[0-9]*$')).show()

+---+------+----------+
| ID|  Name| Mobile_no|
+---+------+----------+
|  2|Lishma|8148190681|
+---+------+----------+



In [0]:
# dbutils.fs.rm("dbfs:/FileStore/sample_excel_csv.xlsx",True)
df = spark.read.csv("dbfs:/FileStore/sample_nihala.csv")
df.show()

+----+----+
| _c0| _c1|
+----+----+
|NULL|null|
|NULL|null|
|  AB|null|
|  CH|null|
|  ID|NAME|
|   1|   A|
|   2|   B|
|   3|   C|
|null|   D|
+----+----+



In [0]:
rdd = sc.textFile("dbfs:/FileStore/sample_nihala.csv").zipWithIndex().filter(lambda a:a[1]>3).map(lambda a:a[0].split(','))
rdd.collect()e

Out[53]: [['ID', 'NAME'], ['1', 'A'], ['2', 'B'], ['3', 'C'], ['', 'D']]

In [0]:
column_rdd = rdd.first()
column_rdd

Out[37]: ['ID', 'NAME']

In [0]:
main_rdd = rdd.filter(lambda a:a!=column_rdd)
df = main_rdd.toDF(column_rdd)
display(df)

ID,NAME
1,A
2,B
3,C
,D


In [0]:
%fs head 'dbfs:/FileStore/sample_csv_format.csv'

ID ,Name,Age
1,A,23
2,B,null
3,C,56
4,null,null
5,null,null

In [0]:
df = spark.read.option("nullvalue","null").option("header",True).csv("dbfs:/FileStore/sample_csv_format.csv")
df.show()
# df_1 = df.select([count(i) for i in df.columns])
df_1 = df.select([count(when(col(i).isNull(),i)).alias(i)for i in df.columns])
df_1.show()

+---+----+----+
|ID |Name| Age|
+---+----+----+
|  1|   A|  23|
|  2|   B|null|
|  3|   C|  56|
|  4|null|null|
|  5|null|null|
+---+----+----+

+---+----+---+
|ID |Name|Age|
+---+----+---+
|  0|   2|  3|
+---+----+---+



In [0]:
%fs head 'dbfs:/FileStore/sample_multi.csv'

ID ,Name,Age,Marks
1,A,20,31|32|34
2,B,21,21|32|43
3,C,22,21|32|11
4,D,23,10|12|12

In [0]:
df = spark.read.option("header",True).option("sep",',').csv("dbfs:/FileStore/sample_multi.csv")
df.show()
df_1 = df.withColumn("phy",split(df.Marks,'\\|')[0]).withColumn("che",split(df.Marks,'\\|')[0]).withColumn("math",split(df.Marks,'\\|')[0]).drop("Marks")
df_1.show()

+---+----+---+--------+
|ID |Name|Age|   Marks|
+---+----+---+--------+
|  1|   A| 20|31|32|34|
|  2|   B| 21|21|32|43|
|  3|   C| 22|21|32|11|
|  4|   D| 23|10|12|12|
+---+----+---+--------+

+---+----+---+---+---+----+
|ID |Name|Age|phy|che|math|
+---+----+---+---+---+----+
|  1|   A| 20| 31| 31|  31|
|  2|   B| 21| 21| 21|  21|
|  3|   C| 22| 21| 21|  21|
|  4|   D| 23| 10| 10|  10|
+---+----+---+---+---+----+



In [0]:
df = spark.read.option("header",True).option("sep",',').csv("dbfs:/FileStore/sample_multi.csv")
df_2 = df.select("ID","Name","Age",explode(split(df.Marks,'\\|')))
df_2.show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1805315907579343>:2
      1 df = spark.read.option("header",True).option("sep",',').csv("dbfs:/FileStore/sample_multi.csv")
----> 2 df_2 = df.select("ID","Name","Age", explode(split(df.Marks,'\\|')))
      3 df_2.show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3023, in DataFrame.select(self, *cols)
   2978 def select(self, *cols: "ColumnOrName") -> "DataFrame":  # type: ignore[misc]
   2979     """Projects a set of expressions and returns a n

In [0]:
1-A-12-2-B-23-3-C-34-4-D-15

In [0]:
dbutils.fs.put('FileStore/tables/pracc.txt','1-A-12-2-B-23-3-C-34-4-D-15')

Wrote 27 bytes.
Out[88]: True

In [0]:
df = spark.read.text("dbfs:/FileStore/tables/pracc.txt")
df1 = df.withColumn("new_value",regexp_replace("value","(.*?\\-){3}","$0,")).drop("value")
df2 = df1.withColumn("new_value1",explode(split(df1.new_value,'-,'))).drop("new_value")
df3 = df2.withColumn("ID", split(df2.new_value1, '-')[0]).withColumn("Name", split(df2.new_value1, '-')[1]).withColumn("Age", split(df2.new_value1, '-')[2]).drop("new_value1")
df3.show()

+---+----+---+
| ID|Name|Age|
+---+----+---+
|  1|   A| 12|
|  2|   B| 23|
|  3|   C| 34|
|  4|   D| 15|
+---+----+---+

